# Heads Up

In this notebook, we start generating random undirected weighted
graphs to not only better approximate real world road network but
also facilitate the feature engineering of nodes and edges

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import mxnet as mx
import logging
from sklearn.metrics import accuracy_score
from utils import plot_g

In [ ]:
NUM_NODE = 100
WEIGHT_MIN = .0
WEIGHT_MAX = 1.

In [ ]:
def generate_low_degree_g(node_size=20, min_out_degree=2, max_out_degree=4, weight_min=WEIGHT_MIN, weight_max=WEIGHT_MAX):
    
    G = nx.Graph()
    G.add_nodes_from(range(0, node_size))
    
    for node in G.nodes:
        
        tmp_nodes = list(G.nodes)
        tmp_nodes.remove(node)
        random.shuffle(tmp_nodes)
        num_of_neighbors = random.randint(min_out_degree, max_out_degree)
#         print(node, out_neighbors)
        
#         G.add_edges_from(map(lambda d:(node, d), out_neighbors))
        
        for tmp_node in tmp_nodes:

            if G.degree(tmp_node) >= max_out_degree:
                # This node has maximum number of neighbors already
                continue

            if G.degree(node) >= num_of_neighbors:
                continue
            
            weight = random.uniform(weight_min, weight_max)
            G.add_edge(node, tmp_node, weight=weight)
        
    return G

In [ ]:
G = generate_low_degree_g()
max_degree = max(G.degree, key=lambda d: d[1])[1]
min_degree = min(G.degree, key=lambda d: d[1])[1]
print(max_degree)
print(min_degree)
print(G.number_of_nodes())
print(G.number_of_edges())
plot_g(G)

In [ ]:
ret = traverse_g(G, 6, depth=3)

In [ ]:
ret

In [ ]:
def traverse_g_helper(layer_2_layer_dict, G, depth, parent_node, cur_node, step):
    
    if step > depth:
        return
    
    layer_2_layer_dict[step-1][cur_node] = filter(lambda edge: edge[1] != parent_node, G.edges(cur_node))
    for edge in G.edges(cur_node):
        
        neighbor = edge[1]
        if neighbor == parent_node:
            continue
        
        print('Step %d: from %d to %d' % (step, cur_node, neighbor))
        traverse_g_helper(layer_2_layer_dict, G, depth, cur_node, neighbor, step+1)
    
def traverse_g(G, src, depth=2):
    
    layer_2_layer_dict = [dict() for _ in range(depth)]
    traverse_g_helper(layer_2_layer_dict, G, depth, -1, src, 1)
    
    return layer_2_layer_dict

In [ ]:
def extract_path(prev, src, dst):
    
    path = []
    u = dst

    while prev[u] != -1:
        path.insert(0, u)
        u = prev[u]
        
    path.insert(0, src)
    return path

def my_dijkstra_path(G, src, dst=None):
    
    prev = [-1 for _ in range(G.number_of_nodes())]
    distance = [float('Inf') for _ in range(G.number_of_nodes())]
    distance[src] = 0
    Q = {}
    intermediate_paths = {}
    
    for node, dist in enumerate(distance):
        Q[node] = dist
        intermediate_paths[node] = []
    
    while len(Q) != 0:
        
        u = min(Q, key=Q.get)
        del Q[u]
        
        for edge in G.edges(u):
            
            
            v = edge[1]
            new_dist = distance[u] + G.get_edge_data(u, v)['weight']
            
#             intermediate_paths[v] += 1
            
            if new_dist < distance[v]:
                distance[v] = new_dist
                Q[v] = new_dist
                prev[v] = u
                
                # extract the tmp path for v here for analysis
                # TODO
                
                intermediate_paths[v].append(extract_path(prev, src, v))
                
    return extract_path(prev, src, dst), intermediate_paths

In [ ]:
for node in G.nodes:

    src, dst = node, 1
    paths_from_src = nx.single_source_dijkstra_path(G, src)

    out_neighbor_chosen_count = {}

    for dst, path in paths_from_src.items():
        if len(path) < 2:
            continue

        assert path[-1] == dst

        out_neighbor_chosen_count[path[1]] = 1 if path[1] not in out_neighbor_chosen_count \
                                                else out_neighbor_chosen_count[path[1]] + 1

    print(src)
    for edge in G.out_edges(src):
        print(edge[1], G.get_edge_data(src, edge[1])['weight'], out_neighbor_chosen_count.get(edge[1], 0))


In [ ]:
plot_g(G)